# Projeto : Mineração e analise de dados provinientes da plataforma Jira

### Requisitos

In [16]:
#!pip install customtkinter
#!pip install python-dotenv
#!pip install text2emotion
#!pip install emoji==1.6.3
#!pip install deep-translator
#!pip install pyvirtualdisplay

### Importações das bibliotecas

In [12]:
import requests
import customtkinter as ctk
from tkinter import messagebox, Toplevel, Checkbutton, IntVar, Canvas, Frame, Scrollbar
import json
import os 
from dotenv import load_dotenv
from urllib.parse import urlparse
import pandas as pd 

In [14]:
import text2emotion as te
from deep_translator import (GoogleTranslator,
                             ChatGptTranslator,
                             MicrosoftTranslator,
                             PonsTranslator,
                             LingueeTranslator,
                             MyMemoryTranslator,
                             YandexTranslator,
                             PapagoTranslator,
                             DeeplTranslator,
                             QcriTranslator,
                             single_detection,
                             batch_detection)

### Chaves de atutenticação

In [78]:
#INSERIR AQUI OS DADOS VALIDOS PARA CONEXÃO E COLETA DOS DADOS
EMAIL = "email@gmail.com"
API_TOKEN = "eJZu_KMdV_tUN0oAgyZD7kuAQEbwJVO7EvmIoaD03dR7vwjXzx1oGg3UoPOs03mC9AF"
PROJECT_URL = "https://projetoxpto.atlassian.net/jira/software/c/projects/cpto/boards/1"
PROJECT_KEY = "XPTO"

### Função de coleta das tarefas

In [38]:
def collect_tasks(jira_domain, project_key, task_type, auth, fields):
    url = f"https://{jira_domain}/rest/api/2/search"
    query = {
        'jql': f'project={project_key} AND issuetype={task_type}',
        'fields': fields
    }
    # print(f"Request URL: {url}")
    # print(f"Request Query: {query}")
    response = requests.get(url, params=query, auth=auth)
    try:
        response.raise_for_status()
    except requests.exceptions.HTTPError as e:
        print(f"Error response: {response.text}")
        raise e
    issues = response.json()['issues']
    
    return issues

### Função de coleta dos campos disponives nas tarefas

In [40]:
def get_available_fields(jira_domain, auth):
    url = f"https://{jira_domain}/rest/api/2/field"
    response = requests.get(url, auth=auth)
    response.raise_for_status()
    fields = response.json()
    return fields

In [41]:
def extract_jira_domain(url: str):
    """
    Extrai o domínio relevante da URL do Jira.
    
    Args:
    url (str): A URL completa do Jira.
    
    Returns:
    str: O domínio relevante do Jira.
    """
    parsed_url = urlparse(url)
    return parsed_url.netloc

### APP principal com interface de interação com usuário

In [29]:
class JiraDataMinerApp(ctk.CTk):

    def __init__(self):
        super().__init__()

        self.title("Jira Data Miner")
        self.geometry("800x800")
        
        self.url_label = ctk.CTkLabel(self, font=('Roboto', 25, "bold"), text="Jira Domain URL:")
        self.url_label.pack(pady=(100,30))
        self.url_entry = ctk.CTkEntry(self, width=400, placeholder_text=PROJECT_URL)
        self.url_entry.pack(pady=(0,50))

        self.key_label = ctk.CTkLabel(self, font=('Roboto', 25, "bold"), text="Project Key:")
        self.key_label.pack(pady=(0,30))
        self.key_entry = ctk.CTkEntry(self, width=400,  placeholder_text=PROJECT_KEY)
        self.key_entry.pack(pady=(0,50))

        self.load_fields_button = ctk.CTkButton(self,
                                 width=200,
                                 height=40,
                                 border_width=0,
                                 corner_radius=8,
                                 text="Load Fields", font=('Roboto', 15, "bold"), command=self.load_fields)
        self.load_fields_button.pack(pady=30)

        self.fields_window = None
        self.fields_vars = {}

    def mine_data(self):
        url = PROJECT_URL
        jira_domain = extract_jira_domain(url)
        project_key = PROJECT_KEY
        auth = (EMAIL, API_TOKEN)

        if not jira_domain or not project_key:
            messagebox.showerror("Error", "Please enter both the Jira Domain URL and Project Key.")
            return

        task_types = []
        if self.epics_switch.get():
            task_types.append(('Epic', collect_tasks))
        if self.user_stories_switch.get():
            task_types.append(('Story', collect_tasks))
        if self.tasks_switch.get():
            task_types.append(('Task', collect_tasks))
        if self.subtasks_switch.get():
            task_types.append(('Sub-task', collect_tasks))
        if self.bugs_switch.get():
            task_types.append(('Bug', collect_tasks))
        if self.enablers_switch.get():
            task_types.append(('Enabler', collect_tasks))

        all_issues = {}
        fields = ','.join(self.selected_fields)
        for task_type, func in task_types:
            #print(task_type)
            tasks = func(jira_domain, project_key, task_type, auth, fields)
            all_issues[task_type] = tasks
            print(f"Collected {len(tasks)} {task_type}(s)")

        save_to_json(all_issues, project_key)
        messagebox.showinfo("Success", f"Data has been successfully mined and saved to {project_key.lower()}_issues.json")

    def show_mining_options(self):
        
        self.epics_switch = ctk.CTkSwitch(self, text="Epics")
        self.epics_switch.pack(pady=5)
        self.user_stories_switch = ctk.CTkSwitch(self, text="User Stories")
        self.user_stories_switch.pack(pady=5)
        self.tasks_switch = ctk.CTkSwitch(self, text="Tasks")
        self.tasks_switch.pack(pady=5)
        self.subtasks_switch = ctk.CTkSwitch(self, text="Sub-tasks")
        self.subtasks_switch.pack(pady=5)
        self.bugs_switch = ctk.CTkSwitch(self, text="Bugs")
        self.bugs_switch.pack(pady=5)
        self.enablers_switch = ctk.CTkSwitch(self, text="Enablers")
        self.enablers_switch.pack(pady=5)
        self.mine_button = ctk.CTkButton(self, 
                                         width=200, 
                                         height=40, 
                                         border_width=0, 
                                         corner_radius=8, 
                                         font=('Roboto', 15, "bold"),
                                         text="Mine Data", command=self.mine_data)
        self.mine_button.pack(pady=30)
    
    def confirm_selection(self):
        selected_fields = [field_id for field_id, var in self.fields_vars.items() if var.get() == 1]
        if not selected_fields:
            messagebox.showerror("Error", "Please select at least one field.")
            return

        self.selected_fields = selected_fields
        self.fields_window.destroy()
        self.show_mining_options()

    def show_fields_selection(self, fields):
        if self.fields_window:
            self.fields_window.destroy()

        self.fields_window = Toplevel(self)
        self.fields_window.title("Select Fields")
        self.fields_window.geometry("500x650")

        canvas = Canvas(self.fields_window)
        canvas.pack(side="left", fill="both", expand=True)

        scrollbar = Scrollbar(self.fields_window, orient="vertical", command=canvas.yview)
        scrollbar.pack(side="right", fill="y")

        scrollable_frame = Frame(canvas)
        scrollable_frame.bind(
            "<Configure>",
            lambda e: canvas.configure(
                scrollregion=canvas.bbox("all")
            )
        )

        canvas.create_window((0, 0), window=scrollable_frame, anchor="nw")
        canvas.configure(yscrollcommand=scrollbar.set)

        for field in fields:
            var = IntVar()
            chk = Checkbutton(scrollable_frame, text=field['name'], variable=var)
            chk.pack(anchor='w')
            self.fields_vars[field['id']] = var

        # Adicionar um Frame adicional para o botão
        button_frame = Frame(scrollable_frame)
        button_frame.pack(fill='x', pady=10)
        self.confirm_button = ctk.CTkButton(button_frame,
                                            width=200, 
                                            height=40, 
                                            border_width=0, 
                                            corner_radius=8, 
                                            font=('Roboto', 15, "bold"),
                                            text="Confirm Selection", command=self.confirm_selection)
        self.confirm_button.pack(pady=20, anchor='center')
    
    def load_fields(self):

        url = PROJECT_URL
        jira_domain = extract_jira_domain(url)
        auth = (EMAIL, API_TOKEN)

        try:
            fields = get_available_fields(jira_domain, auth)
        except requests.exceptions.HTTPError as e:
            messagebox.showerror("Error", f"Failed to load fields: {e}")
            return

        self.show_fields_selection(fields)


### Função responsável por salvar os dados coletados em um arquivo Json

In [32]:
def save_to_json(data, project_key):
    filename=f'{project_key.lower()}_issues.json'
    with open(filename, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=4)

### Chamada para execuçãoo do app com interface de interação com o usuário

In [48]:
if __name__ == "__main__":
    app = JiraDataMinerApp()
    app.mainloop()

Collected 50 Sub-task(s)


In [ ]:
#Importar Json de dados gerado

In [50]:
filename = f'{PROJECT_KEY.lower()}_issues.json'
df = pd.read_json(filename)

In [52]:
key = []
status = []
assignee_id = []
assignee_name = []
originalEstimate = []
remainingEstimate = []
timeSpent = []
text_br = []
text_en = []
emotion = []
happy = []
angry = []
surprise = []
sad = []
fear = []

### Obtem informações textuais de comentários nas tarefas

In [55]:
def getComments(taskComments):
    i = 0
    comments = " "
    while i < len(taskComments):
        comments += taskComments[i]['body']
        i += 1

    return comments

### Realiza o pós processamento dos dados obtidos e analisa o sentimento contido nas mensagens textuais

In [57]:
i = 0
#def dataProcess(df):
while i < len(df['Sub-task']):
    if df['Sub-task'][i]['fields']['assignee'] is not None:
        task_key = df['Sub-task'][i]['key']
        task_status = df['Sub-task'][i]['fields']['status']['name']
        task_assignee_id = (df['Sub-task'][i]['fields']['assignee']['accountId'])
        task_assignee_name = df['Sub-task'][i]['fields']['assignee']['displayName']
        task_description = df['Sub-task'][i]['fields']['description']
        task_summary = df['Sub-task'][i]['fields']['summary']
        task_comments = df['Sub-task'][i]['fields']['comment']['comments']
        
        if df['Sub-task'][i]['fields']['timetracking'] :
            timetracking = df['Sub-task'][i]['fields']['timetracking']
            task_originalEstimate = df['Sub-task'][i]['fields']['timetracking']['originalEstimateSeconds']
            task_remainingEstimate = df['Sub-task'][i]['fields']['timetracking']['remainingEstimateSeconds']
            task_timeSpent = 0 if len(timetracking)<6 else (timetracking['timeSpentSeconds'])

        else :
            task_originalEstimate = 0
            task_remainingEstimate = 0
            task_timeSpent = 0
        
        textComments = getComments(task_comments)
        
        task_text_br = (('d' if task_summary is None else task_summary) + " " +
                        ('d' if task_description is None else task_description) + " " +
                        ('d' if textComments is None else textComments))
        task_text_en = GoogleTranslator(source='auto', target='en').translate(task_text_br)
        task_emotion = te.get_emotion(task_text_en)
        task_happy = task_emotion['Happy']
        task_angry = task_emotion['Angry']
        task_surprise = task_emotion['Surprise']
        task_sad = task_emotion['Sad']
        task_fear = task_emotion['Fear']
        
        
        key.append(task_key)
        status.append(task_status)
        assignee_id.append(task_assignee_id)
        assignee_name.append(task_assignee_name)
        originalEstimate.append(task_originalEstimate)
        remainingEstimate.append(task_remainingEstimate)
        timeSpent.append(task_timeSpent)
        text_br.append(task_text_br)
        text_en.append(task_text_en)
        happy.append(task_happy)
        angry.append(task_angry)
        surprise.append(task_surprise)
        sad.append(task_sad)
        fear.append(task_fear)
        emotion.append(task_emotion)
        
        
    else:
        print("Not assignee " + df['Sub-task'][i]['key'])
        
    i += 1

### Cria objeto compilando dados processados

In [58]:
data = {
'key' : key,
'status' : status,
'assignee_id' : assignee_id,
'assignee_name' : assignee_name,
'originalEstimate' : originalEstimate,
'remainingEstimate' : remainingEstimate,
'timeSpent' : timeSpent,
'text_br' : text_br,
'text_en' : text_en,
'happy' : happy,
'angry' :angry,
'surprise' : surprise,
'sad' : sad,
'fear' : fear,
'emotion' : emotion
}

### Imprime resultado do processamento de emoções

In [62]:
emotion

[{'Happy': 0.0, 'Angry': 0.0, 'Surprise': 0.0, 'Sad': 0.0, 'Fear': 1.0},
 {'Happy': 0, 'Angry': 0, 'Surprise': 0, 'Sad': 0, 'Fear': 0},
 {'Happy': 0, 'Angry': 0, 'Surprise': 0, 'Sad': 0, 'Fear': 0},
 {'Happy': 0.0, 'Angry': 0.0, 'Surprise': 0.0, 'Sad': 0.0, 'Fear': 1.0},
 {'Happy': 0, 'Angry': 0, 'Surprise': 0, 'Sad': 0, 'Fear': 0},
 {'Happy': 0, 'Angry': 0, 'Surprise': 0, 'Sad': 0, 'Fear': 0},
 {'Happy': 1.0, 'Angry': 0.0, 'Surprise': 0.0, 'Sad': 0.0, 'Fear': 0.0},
 {'Happy': 0.0, 'Angry': 0.0, 'Surprise': 0.0, 'Sad': 0.0, 'Fear': 1.0},
 {'Happy': 0, 'Angry': 0, 'Surprise': 0, 'Sad': 0, 'Fear': 0},
 {'Happy': 0.21, 'Angry': 0.0, 'Surprise': 0.0, 'Sad': 0.07, 'Fear': 0.71},
 {'Happy': 0.0, 'Angry': 0.0, 'Surprise': 0.0, 'Sad': 0.0, 'Fear': 1.0},
 {'Happy': 0.0, 'Angry': 0.0, 'Surprise': 0.0, 'Sad': 0.4, 'Fear': 0.6},
 {'Happy': 0.5, 'Angry': 0.0, 'Surprise': 0.5, 'Sad': 0.0, 'Fear': 0.0},
 {'Happy': 0.0, 'Angry': 0.0, 'Surprise': 0.0, 'Sad': 0.5, 'Fear': 0.5},
 {'Happy': 0, 'Angry': 0,

### Cria dataframe com dados processados

In [64]:
dfData = pd.DataFrame(data)

In [66]:
dfData

,key,status,assignee_id,assignee_name,originalEstimate,remainingEstimate,timeSpent,text_br,text_en,happy,angry,surprise,sad,fear,emotion
0,CSTONE-292,Em análise,62b3a9f1566f3e7b0a902aec,Diogo Marassi,18000,7200,10800,"Minerar, extrair mensagens e tags dos commits ...","Mine, extract messages and tags from commits a...",0.00,0.00,0.00,0.00,1.00,"{'Happy': 0.0, 'Angry': 0.0, 'Surprise': 0.0, ..."
1,CSTONE-291,Em andamento,712020:f1b2bd62-8419-4287-b99c-9e04cf02a207,Gabriel Martins,7200,7200,0,Começar a aplicação do Jira no Django Criar o ...,Starting your Jira app on Django Create your J...,0.00,0.00,0.00,0.00,0.00,"{'Happy': 0, 'Angry': 0, 'Surprise': 0, 'Sad':..."
2,CSTONE-290,Tarefas pendentes,70121:8dbbb0cf-e4a9-4994-b9f1-dee96394b311,Lucas Lopes,7200,7200,0,Teste para disagreementde LLMs Testar as LLMs ...,Test for disagreement of LLMs Test the LLMs of...,0.00,0.00,0.00,0.00,0.00,"{'Happy': 0, 'Angry': 0, 'Surprise': 0, 'Sad':..."
3,CSTONE-289,Em andamento,712020:f1b2bd62-8419-4287-b99c-9e04cf02a207,Gabriel Martins,14400,14400,0,Criação do model para as issues do Jira Criar ...,Creating a model for Jira issues Create a mode...,0.00,0.00,0.00,0.00,1.00,"{'Happy': 0.0, 'Angry': 0.0, 'Surprise': 0.0, ..."
4,CSTONE-288,Em andamento,70121:8dbbb0cf-e4a9-4994-b9f1-dee96394b311,Lucas Lopes,10800,3600,7200,Pesquisa de como aplicar Docker na API PPesqis...,Research on how to apply Docker to the API PPe...,0.00,0.00,0.00,0.00,0.00,"{'Happy': 0, 'Angry': 0, 'Surprise': 0, 'Sad':..."
5,CSTONE-287,Em andamento,70121:8dbbb0cf-e4a9-4994-b9f1-dee96394b311,Lucas Lopes,14400,10800,3600,Criação de um subprocesso para usar o código R...,Creating a subprocess to use Rust code Rust co...,0.00,0.00,0.00,0.00,0.00,"{'Happy': 0, 'Angry': 0, 'Surprise': 0, 'Sad':..."
6,CSTONE-286,Em análise,70121:8dbbb0cf-e4a9-4994-b9f1-dee96394b311,Lucas Lopes,7200,0,7200,Refatoração do código Rust para API Refatorar ...,Refactoring Rust code for API Refactoring Rust...,1.00,0.00,0.00,0.00,0.00,"{'Happy': 1.0, 'Angry': 0.0, 'Surprise': 0.0, ..."
7,CSTONE-285,Tarefas pendentes,62b3a9f1566f3e7b0a902aec,Diogo Marassi,18000,18000,0,Estudar e Implementar código para classificaçã...,Study and Implement code for classification fr...,0.00,0.00,0.00,0.00,1.00,"{'Happy': 0.0, 'Angry': 0.0, 'Surprise': 0.0, ..."
8,CSTONE-284,Concluído,62b3a9f1566f3e7b0a902aec,Diogo Marassi,7200,0,7200,Criar repositório stnl-nfrclassification A tar...,Create stnl-nfrclassification repository The t...,0.00,0.00,0.00,0.00,0.00,"{'Happy': 0, 'Angry': 0, 'Surprise': 0, 'Sad':..."
9,CSTONE-283,Em andamento,62b3a9f1566f3e7b0a902aec,Diogo Marassi,18000,7200,10800,Estudar e criar base para implementar a classi...,Study and create a basis for implementing clas...,0.21,0.00,0.00,0.07,0.71,"{'Happy': 0.21, 'Angry': 0.0, 'Surprise': 0.0,..."
